## Yelp Data Initial Exploration - Business Data

In [ ]:
## preps
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
print(1e7)

In [ ]:
def plotTopFreq(dat, var, top_n, title):
    #cat_data.categories
    Categories={}
    for cat in dat[var].values:
#     print(cat)
        if cat != None:
            all_categories= cat.split(",")
            all_categories = [x.strip() for x in all_categories]
        for x in all_categories:
            try :
                Categories[x] =Categories[x]+1
            except:
                Categories[x]=1
    top_categories = pd.DataFrame.from_dict(data= Categories,orient="index")
    top_categories.reset_index(inplace=True)
    top_categories.columns = ['category', 'occurance']

    x_val=top_categories.sort_values("occurance")[::-1][:top_n].occurance.values
    labels=top_categories.sort_values("occurance")[::-1][:top_n].category.values
    series = pd.Series(x_val, index=labels, name=title)
    series.plot.pie(figsize=(10, 10),startangle=90)
    

## Business Data preview

In [ ]:
# load and preview the data
business_data = []
with open("/Users/kalhan/Downloads/yelp_academic_dataset_business.json") as f:
    for line in f:
        business_data.append(json.loads(line))
business_df = pd.DataFrame.from_dict(business_data)
business_df.head()


In [ ]:
business_df.info()

In [ ]:
# "name" column name is ambiguous with df.name - change it
business_df = business_df.rename(columns = {'name': 'business_name'})

In [ ]:
## Top 15 business ategories
plotTopFreq(business_df,"categories",15,"Top business types")

In [ ]:
## top cities
plotTopFreq(business_df,"city",15,"Top cities")

### We subset restaurants:


In [ ]:
restaurants = (business_df["categories"].str.find("Restaurants")>=0)
bars = (business_df["categories"].str.find("Bars")>=0)
food = (business_df["categories"].str.find("Food")>=0)
food_restaurants_bar = [a or b or c for a, b,c in zip(restaurants, bars, food)]

In [ ]:
restaurants.sum()

In [ ]:
restaurants_bar = [a or b for a, b in zip(restaurants, bars)]
np.sum(restaurants_bar)

* __We do not need to keep the business universe too broad, since that way the review matrix will be even more sparse. Instead we first only focus on "restaurants"__

In [ ]:
foods_df = business_df[restaurants]
print(foods_df.shape)
foods_df.head()

- __Second, We focus on cities where the reviews per business is high__

In [ ]:
plt.subplot(1,2,1)
plotTopFreq(foods_df,"categories",10,"Top Business")
plt.subplot(1,2,2)
plotTopFreq(foods_df,"city",10,"Top cities")

In [ ]:
## restaurant review counts by city
review_count = foods_df.groupby("city", as_index=False)["review_count"].agg(sum).sort_values(by = "review_count", ascending = False)
top_reviews = review_count.iloc[:15]
top_reviews = top_reviews.set_index("city")
top_reviews.plot.bar()
plt.title("top review counts")

In [ ]:
top_reviews

## Finalize our candidate cities:

In [ ]:
## For simplicity, we choose the top 5 cities by review counts
top_reviews.iloc[:5]

In [ ]:
### Top review counts per restaurant
top_avg_reviews = foods_df.groupby("city", as_index=False)["review_count"].median().sort_values("review_count", ascending = False)
top_avg_reviews.loc[top_avg_reviews["city"].isin(top_reviews.index.values)].plot.bar(x = "city", y = "review_count")
plt.title("Median reviews per retaurants for those cities in top 15 by total reviews")

In [ ]:
## we keep the top candidate cities that have high median reviews per restaurant
candidates = top_avg_reviews.loc[top_avg_reviews["city"].isin(top_reviews.iloc[:5].index.values)]
print("Avg reviews per restaurant:")
print(candidates)

In [ ]:
df_res_business = foods_df[foods_df["city"].isin(candidates["city"])]
df_res_business.to_csv("business_data_subset.csv", index = None)

In [ ]:
df_res_business.head()

In [ ]:
df_res_bus = pd.read_csv("business_data_subset.csv", index_col= None)
df_res_bus_ids = df_res_bus.business_id.unique()
df_res_bus.head()

In [ ]:
print(df_res_bus.shape)
print(df_res_bus.groupby(["city"])["business_id"].size().sort_values(ascending = False).plot.bar())
plt.title("number of restaurants")

In [ ]:
print("*"*80)
print("Total Reviews:")
print(df_res_bus.groupby(["city"])["review_count"].agg(sum).sort_values(ascending = False).plot.bar())
print("*"*80)
print("Reviews per restaurant:")
df_res_bus.groupby(["city"])["review_count"].describe().sort_values("50%", ascending = False) 

## Review Data

In [ ]:
## Review data for restaurants
## This will take a long time
'''
# THIS CELL HAS ALREADY BEEN EXECUTED
review_data_samp = []
count = 0
count_cap = 1e7 #### TODO, increse this cap to atleast 1e8
count = 0
with open("/Users/kalhan/Downloads/yelp_academic_dataset_review.json") as f:
    for i,line in enumerate(f):
        print(i)
        if i > count_cap:
            break 
        if json.loads(line)["business_id"] in df_res_bus_ids:
            review_data_samp.append(json.loads(line))   
review_df_samp = pd.DataFrame.from_dict(review_data_samp)
review_df_samp.to_csv("reviews_data_subset.csv", index=None)
'''

In [ ]:
reviews_df = pd.read_csv("reviews_data_subset.csv")
reviews_df.head()
reviews_df.head()


In [ ]:
print(reviews_df.shape)


df_merge = pd.merge(reviews_df, df_res_bus[['city','categories', 'business_id',"review_count"]], how = "outer", on ="business_id")
df_merge.head()


In [ ]:
df_merge.isnull().sum()

In [ ]:
df_merge.dropna(inplace=True)

In [ ]:
df_merge.city.unique()

In [ ]:
x = df_merge.groupby(["city","user_id"]).size().reset_index()
print("users' reviews summary statistics")
x.groupby("city").describe()

In [ ]:
x.groupby("city").describe()[(0, "mean")].sort_values(ascending = False).plot.bar()
plt.title("Average reviews per User")

## User Data

In [ ]:
## get a list of user ids that had reviews for our candidate cities
users_ids_subset = df_merge.user_id.unique().tolist()

In [ ]:
print(len(users_ids_subset))

In [ ]:
print(df_merge.shape)
df_merge_filtered_philly = df_merge[df_merge["city"]=="Philadelphia"]
print(df_merge_filtered_philly.shape)

In [ ]:
users_ids_subset_philly = df_merge_filtered_philly.user_id.unique().tolist()
print(len(users_ids_subset_philly))
users_ids_subset_philly = set(users_ids_subset_philly)

WE HAVE AROUDN 200K UNIQUE USERS FOR REVIEWS OF PHILLY ITSELF.

In [ ]:
user_data_samp = []
with open("/Users/kalhan/Downloads/yelp_academic_dataset_user.json") as f:
    for line in f:
        if json.loads(line)["user_id"] in users_ids_subset_philly:
            user_data_samp.append(json.loads(line))
            
user_data_samp = pd.DataFrame.from_dict(user_data_samp)
user_data_samp.to_csv("user_df.csv", index=None)

In [ ]:
len(user_data_samp)

In [ ]:
user_data_samp.head()


In [ ]:
user_data_samp.info()